In [1]:
import sys
import os, os.path

sys.path.append(os.path.join(os.getcwd() ,'/modules'))
root_path = "C:/git/Springboard-Public/Capstone Project 2/"
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = "/content/drive/My Drive/Capstone Project 2/"

print('Current Working Dir: ', os.getcwd())
print('Root Path: ', root_path)

# We need to set the working directory since we are using relative paths from various locations
if os.getcwd() != root_path:
  os.chdir(root_path)

Current Working Dir:  C:\git\Springboard-Public\Capstone Project 2\notebooks
Root Path:  C:/git/Springboard-Public/Capstone Project 2/


In [2]:
import numpy as np
from datetime import datetime
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from modules.lib.ChextXRayImages import *
from modules.models.CustomPneumonia import CustomPneumoniaNN

from PIL import Image
import copy

import torch.optim as optim
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, ToPILImage
import torchvision.models as models

from torchsummary import summary

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

%matplotlib inline

In [3]:
force_cpu = True
device = torch.device('cuda' if ~force_cpu and torch.cuda.is_available() else 'cpu')
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(f'Working on device={device}')


Working on device=cuda


In [4]:
loaders = Loaders()
batch_size=16
val_percent=0.15
number_images = 1000
train_loader, val_loader = loaders.getDataTrainValidateLoaders(batch_size=batch_size, 
                                                                        val_percent=val_percent, 
                                                                        n_random_rows=number_images)
print(f'Number of Training Batches: {len(train_loader):,}')
print(f'Number of Validation Batches: {len(val_loader):,}')
print(f'Number of Training Images: {len(train_loader) * batch_size:,}')
print(f'Number of Validation Images: {len(val_loader) * batch_size:,}')

Number of Training Batches: 54
Number of Validation Batches: 10
Number of Training Images: 864
Number of Validation Images: 160


In [5]:
net = CustomPneumoniaNN()

net = nn.DataParallel(net)
net.to(device)

summary(net, (1, 320, 320))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 512, 320, 320]          13,312
       BatchNorm2d-2        [-1, 512, 320, 320]           1,024
         MaxPool2d-3        [-1, 512, 160, 160]               0
            Conv2d-4        [-1, 256, 160, 160]       1,179,904
       BatchNorm2d-5        [-1, 256, 160, 160]             512
         MaxPool2d-6          [-1, 256, 80, 80]               0
            Conv2d-7           [-1, 64, 80, 80]         147,520
       BatchNorm2d-8           [-1, 64, 80, 80]             128
         MaxPool2d-9           [-1, 64, 40, 40]               0
           Linear-10                 [-1, 1024]     104,858,624
          Dropout-11                 [-1, 1024]               0
           Linear-12                  [-1, 512]         524,800
          Dropout-13                  [-1, 512]               0
           Linear-14                   

In [6]:
learning_rate = 1e-3
num_epochs = 20
# torch.set_num_threads(1)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)#, weight_decay=0.9)

In [7]:
train_accuracy_index = []
train_acc, train_total, train_correct = 0, 0, 0
val_accuracy_index = []
val_acc, val_total, val_correct = 0, 0, 0
test_accuracy_index = []
test_acc, test_total, test_correct = 0, 0, 0
losses_index = []
for epoch in range(num_epochs):  # loop over the dataset multiple times
    start_time = datetime.now()
    net.train()
    running_loss = 0.0
    epoch_loss = 0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data['img'], data['labels']
        # move data to device GPU OR CPU
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        
        #Get Training Accuracty
        # We are using BCEWithLogitsLoss for out loss
        # In this loss funciton, each label gets the sigmoid (inverse of Logit) before the CE loss
        # So our model outputs the raw values on the last FC layer
        # This means we have to apply sigmoid to our outputs to squash them between 0 and 1
        # We then take values >= .5 as Positive and < .5 as Negative
        predicted = outputs.data
        predicted = torch.sigmoid(predicted) 
        predicted[predicted >= 0.5] = 1 # assign 1 label to those with less than 0.5
        predicted[predicted < 0.5] = 0 # assign 0 label to those with less than 0.5
        train_batch_size, train_label_count = labels.shape
        train_acc = float((predicted == labels).sum()) / float((train_batch_size * train_label_count))
        train_accuracy_index.append(train_acc)

        #loss, back prop and update params
        loss = criterion(outputs, labels)#.float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss = epoch_loss / len(train_loader)
    time_elapsed = datetime.now() - start_time
    losses_index.append(epoch_loss)
    
    # Validation set
    net.eval()
    with torch.no_grad():
      for data in val_loader:
          inputs, labels = data['img'], data['labels']
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = net(inputs)
          predicted = outputs.data
          predicted = torch.sigmoid(predicted) 
          predicted[predicted >= 0.5] = 1 
          predicted[predicted < 0.5] = 0 
          val_batch_size, val_label_count = labels.shape
          val_acc = float((predicted == labels).sum()) / float((val_batch_size * val_label_count))
          val_accuracy_index.append(val_acc)
   
    
    print(f'Epoch [{epoch+1}/{num_epochs}], \
          Epoch Loss: {epoch_loss:.4f} \
          Training Accuracy: {train_acc:.4f}  \
          Validation Accuracy: {val_acc:.4f} - (time={time_elapsed})')

RuntimeError: CUDA out of memory. Tried to allocate 3.13 GiB (GPU 0; 8.00 GiB total capacity; 3.53 GiB already allocated; 2.41 GiB free; 3.54 GiB reserved in total by PyTorch)